# Capstone project - Recommending a location for a new hotel in Toronto

## Enviornment preperation

In [2]:
#Prepare dependencies
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import os

#Ignore version and other warnings
#This will not stop exeptions
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Loading the data

In [11]:
#Scraping from Wikipedia
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki=pd.read_html(url)
neighborhoods = wiki[0][:]

#Load the Geospatial_data into a new dataframe
df1 = pd.read_csv('https://cocl.us/Geospatial_data')

## Data preprocessing

In [12]:
#remove any Not Assigned Borough records
cond = neighborhoods['Borough']=='Not assigned' 
neighborhoods.drop(neighborhoods[cond].index, inplace = True)

# Identify duplicate neighbourhoods for postcodes
nh = neighborhoods.groupby('Postcode').count()

# Create a list of postalcodes. Will be used for the iteration 
nharr = neighborhoods.iloc[:,0].unique()

#Create a new dataframe that will contain the final data
df = pd.DataFrame(columns = ['Postalcode', 'Borough', 'Neighborhood']) 


#Iterate through the different postcodes
for n in nharr:
    alln=neighborhoods.ix[neighborhoods['Postcode'] == n]   
    
    if alln.shape[0] >1:
        #More the one record exist for the current postal code, so we'll create a new string that will gather all the neighbourhoods for that code
        str=''
        for x in range(alln.shape[0]):
            if str=='':
                str=alln['Neighbourhood'].iloc[x]
            else:
                str=str + ", " + alln['Neighbourhood'].iloc[x]
        
        #Add this postal code to the final df
        df.loc[len(df)] = [alln['Postcode'].iloc[x], alln['Borough'].iloc[x], str]

    else:
        df.loc[len(df)] = [alln['Postcode'].iloc[0], alln['Borough'].iloc[0], alln['Neighbourhood'].iloc[0]]
 


## Joining the geospatial data to the dataframe using the Geospatial_data.csv 

In [13]:
#Merge the dataframes on the postal code columns
mergedf = pd.merge(df, df1, left_on='Postalcode', right_on='Postal Code')
mergedf = mergedf.drop('Postal Code', 1)

## Map representation of Toronto

In [15]:
#Fetch only Toronto's boroughs
TorontoData = mergedf[mergedf['Borough'].str.contains("Toronto")]
latitude = TorontoData['Latitude'].iloc[0]
longitude = TorontoData['Longitude'].iloc[0]
neighborhood_name = TorontoData['Neighborhood'].iloc[0]
# create map of Toronto using Totonto's df
TorontoMap = folium.Map(location=[latitude, longitude], zoom_start=14)


# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoData['Latitude'], TorontoData['Longitude'], TorontoData['Borough'], TorontoData['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoMap)  
    
TorontoMap

## Query the foursquare DB for Toronto

In [16]:
neighborhood_latitude = latitude # neighborhood latitude value
neighborhood_longitude = longitude # neighborhood longitude value

neighborhood_name = neighborhood_name # neighborhood name
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'B1P43XH4DFXGZOUDTVWNRSZ2SVUGPNWAULXUP5BO3WR5KSTG' # your Foursquare ID
CLIENT_SECRET = 'YUEPDFH4AT4UQJUY0QRLEFVMMPVHB55KKAOF0R4APMM3RQYG' # your Foursquare Secret
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=TorontoData['Neighborhood'],
                                   latitudes=TorontoData['Latitude'],
                                   longitudes=TorontoData['Longitude']
                                  )


Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [20]:
Toronto_venues.groupby('Neighborhood').count()
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)


## Cluster Neighborhoods

In [21]:
# set number of clusters
kclusters = 10

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoData

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head(50) # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Restaurant,Theater,Yoga Studio
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Bookstore,Sporting Goods Shop,Bubble Tea Shop,Pizza Place,Bakery
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Gastropub,Beer Bar,Breakfast Spot,Bakery,Clothing Store
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,9,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Farmers Market,Steakhouse,Seafood Restaurant,Café,Park
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Fried Chicken Joint,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Bar,Spa
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Convenience Store,Coffee Shop
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,2,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Asian Restaurant,Restaurant,Burger Joint
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0,Supermarket,Pharmacy,Bakery,Gym / Fitness Center,Music Venue,Middle Eastern Restaurant,Café,Brewery,Bar,Bank
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,2,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Bakery,Sports Bar,Pizza Place


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']): 
    label = folium.Popup('{}'.format(poi) + ' Cluster ' + '{}'.format(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusters types

In [23]:
for x in range(Toronto_merged['Cluster Labels'].max() +1):
    print ("Cluster " + '{}'.format(x))
    print ("--------------------")
    print(Toronto_merged.loc[Toronto_merged['Cluster Labels'] == x, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].head(10) )
    print ('\n \n')

Cluster 0
--------------------
         Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
31  West Toronto               0           Supermarket              Pharmacy   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
31                Bakery  Gym / Fitness Center           Music Venue   

        6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
31  Middle Eastern Restaurant                  Café               Brewery   

   9th Most Common Venue 10th Most Common Venue  
31                   Bar                   Bank  

 

Cluster 1
--------------------
            Borough  Cluster Labels 1st Most Common Venue  \
47     East Toronto               1                  Park   
67  Central Toronto               1                   Gym   
74  Central Toronto               1        Sandwich Place   
79  Central Toronto               1        Sandwich Place   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venu

In [24]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Hotel,Asian Restaurant,Restaurant,Burger Joint
1,2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Bakery,Farmers Market,Steakhouse,Seafood Restaurant,Café,Park
2,2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Performing Arts Venue,Coffee Shop,Café,Yoga Studio,Caribbean Restaurant,Stadium,Sandwich Place,Burrito Place,Restaurant
3,2,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
4,2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Lounge,Boutique,Bar
5,2,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Café,Bakery,Pub,Pizza Place,Pet Store,Breakfast Spot,Sandwich Place
6,2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Fried Chicken Joint,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Bar,Spa
7,2,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Arts & Crafts Store
8,2,Christie,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Convenience Store,Coffee Shop
9,2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


##  Look for locations where the type of business is commonly located

In [97]:
buisness_type = 'Hotel'

common = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['2nd Most Common Venue'] == 'Hotel' ]
 
common= common.append(neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['3rd Most Common Venue'] == 'Hotel'])

common= common.append(neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['4th Most Common Venue'] == 'Hotel'])
common= common.append(neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['5th Most Common Venue'] == 'Hotel'])

common


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,1,Davisville North,Gym,Hotel,Pizza Place,Convenience Store,Clothing Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Entertainment
19,2,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Bakery,Sports Bar,Pizza Place
10,2,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Steakhouse,Italian Restaurant,Seafood Restaurant,Gym,Gastropub
14,2,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Bakery
16,2,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Deli / Bodega,Bar,Burger Joint,Gym,Asian Restaurant
32,2,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Restaurant,Hotel,Beer Bar,Seafood Restaurant,Italian Restaurant,Fast Food Restaurant,Cocktail Bar,Cheese Shop
31,2,St. James Town,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Gastropub,Beer Bar,Breakfast Spot,Bakery,Clothing Store


## Analyzing the findings

It is possible to identify that hotels are most likely found on cluster 2...

Analizing this cluster reveals that the most common venues are coffee shops, restaurants and bars, which are most likely to be found by hotels.




In [53]:
#Focusing on cluster 2
cluster = Toronto_merged.loc[Toronto_merged["Cluster Labels"]==2]

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster['Latitude'], cluster['Longitude'], cluster['Neighborhood'], cluster['Cluster Labels']): 
    label = folium.Popup('{}'.format(poi) + ' Cluster ' + '{}'.format(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Find potential seaside locations from cluster 2 in which hotels are not in the 5 most common venues

In [119]:
potential =neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted["Cluster Labels"]==2]
potential = pd.concat([potential, common]).loc[potential.index.symmetric_difference(common.index)]
potential =potential.loc[potential["Cluster Labels"]==2]
potential_hoods = potential['Neighborhood']
potentialData = mergedf[mergedf['Neighborhood'].isin(potential_hoods)]
potentialData

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
43,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
54,M4M,East Toronto,Studio District,43.659526,-79.340923


In [129]:
latitude = potentialData['Latitude'].iloc[0]
longitude = potentialData['Longitude'].iloc[0]
neighborhood_name = potentialData['Neighborhood'].iloc[0]
# create map of Toronto using potentialData's df
PMap = folium.Map(location=[latitude, longitude], zoom_start=14)


# add possible location markers to the map
 
for lat, lng, borough, neighborhood in zip(potentialData['Latitude'], potentialData['Longitude'], potentialData['Borough'], potentialData['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=1).add_to(PMap)
    
    
PMap

# Results

At this point we have 6 possible locations that resides by the sea:

* Business Reply Mail Processing Centre 969 Eastern, East Toronto

* Studio District, East Toronto

* Harbourfront, Regent Park, Downtown Toronto

* Berczy Park, Downtown Toronto

* Brockton, Exhibition Place, Parkdale Village, West Toronto

* CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara, Downtown Toronto
